In [4]:
import os
import pandas as pd

In [2]:
def load_imbalance_prices(file: str) -> pd.Series:
    df = pd.read_csv(file, sep=";", decimal=",")
    t = pd.to_datetime(df["Datum"] + " " + df["von"], format='%d.%m.%Y %H:%M') 
    df["datetime"] = pd.date_range(t.iloc[0], t.iloc[-1], freq="15Min") # avoid repeated indices from daylight saving time change
    df = df.set_index("datetime")
    return df

In [5]:
def load_results(dir):
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [25]:
def calculate_imbalance_cost(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    df["imb"] = -(df["power_opt"] - df["power_sim"]) * (1 / 60) * 1e-6 # MWh
    
    # BESS under-supply
    df_u = df.loc[df.imb > 0]
    cost_u = sum(df_u["imb"] * df_u["reBAP unterdeckt"])
    
    # BESS over-supply
    df_o = df.loc[df.imb < 0]
    cost_o = sum(df_o["imb"] * df_o["reBAP ueberdeckt"])

    return cost_u + cost_o

In [26]:
imb_price = load_imbalance_prices("data/balancing_energy_prices/reBAP_2021.csv")

In [27]:
dir = "results/full_year/"
res = load_results(dir)

In [28]:
calculate_imbalance_cost(res["2021 LP fec=1.0 r=1.0 eff=0.95"], imb_price)

48.94987480621723